In [1]:
from src import get_azure_openai_model, get_azure_openai_chat_model, create_vector_store

model = get_azure_openai_chat_model()
vector_store = create_vector_store()

c:\Users\newac\OneDrive\Desktop\Master\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load PDFs
from langchain_community.document_loaders import PyPDFLoader
import os

folder_path = "papers"
pdf_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]

pages = []
for pdf_file in pdf_files:
    loader = PyPDFLoader(os.path.join(folder_path, pdf_file))
    async for page in loader.alazy_load():
        pages.append(page)

In [3]:
# add pdf pages to vector store
_ = vector_store.add_documents(documents=pages)

# Retrival
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

#### Basic setup for langsmith experiments 

In [14]:
import os

subscription_key = os.getenv("AZURE_OPENAI_API_KEY")

In [15]:
from langsmith import wrappers, Client
from pydantic import BaseModel, Field
from openai import OpenAI

client = Client()
openai_client = wrappers.wrap_openai(OpenAI(api_key=subscription_key))

In [ ]:
# Create a dataset [refrence from langsmith page]
import pandas as pd

# Read the CSV file
csv_file_path = os.path.join(folder_path, 'hpb7answers.csv')
df = pd.read_csv(csv_file_path, encoding='latin1')

# Ensure inputs and outputs are in the correct format
examples = [(input_text, output_text) for input_text, output_text in zip(df['Question'].tolist(), df['Answer'].tolist())]

inputs = [{"question": input_prompt} for input_prompt, _ in examples]
outputs = [{"answer": output_answer} for _, output_answer in examples]


# Programmatically create a dataset in LangSmith
dataset = client.create_dataset(
	dataset_name = "Sample dataset2",
	description = "A sample dataset in LangSmith."
)

# Add examples to the dataset
client.create_examples(inputs=inputs, outputs=outputs, dataset_id=dataset.id)

In [ ]:
# Create a dataset [refrence from langsmith page]
import pandas as pd

# Read the CSV file
csv_file_path = os.path.join(folder_path, 'hpb7answers.csv')
df = pd.read_csv(csv_file_path, encoding='latin1')

# Ensure inputs and outputs are in the correct format
examples = [{'input': input_text, 'output': output_text} for input_text, output_text in zip(df['Question'].tolist(), df['Answer'].tolist())]

inputs = [examples[i]["input"] for i in range(len(examples))]
outputs = [examples[i]["output"] for i in range(len(examples))]


# Programmatically create a dataset in LangSmith
dataset = client.create_dataset(
	dataset_name = "Sample dataset",
	description = "A sample dataset in LangSmith."
)

# Add examples to the dataset
client.create_examples(inputs=examples, dataset_id=dataset.id)

In [68]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Define the application logic for evaluation

# TO-DO: Evaluate the self-eval model

 

def target(inputs: dict) -> dict:
    question = inputs["input"]
    # Prompt
    system = """Answer the following question accurately"""
    grade_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system),
            ("human", "{question}"),
        ]
    )

    retrieval_grader = grade_prompt | model | StrOutputParser()
   
    docs = retriever.get_relevant_documents(question)
    return {"response": retrieval_grader.invoke({"question": question, "document": docs})}

    
        

In [67]:
target(inputs)

{'response': 'Harry accuses Remus Lupin of being a coward in "Harry Potter and the Deathly Hallows." This accusation arises during a heated conversation about Lupin\'s decision to leave Tonks and their unborn child to join Harry and the others in the fight against Voldemort. Harry feels that Lupin is abandoning his family out of fear for their safety, which he interprets as cowardice. Harry\'s frustration stems from his own experiences with loss and sacrifice, and he believes that fighting against evil is a responsibility that should not be avoided, even at great personal cost. This moment highlights the themes of bravery, sacrifice, and the complexities of personal choices in the face of danger.'}

In [ ]:
# Define instructions for the LLM judge evaluator
instructions = """Evaluate Answer against Ground Truth for conceptual similarity and classify true or false: 
- False: No conceptual match and similarity
- True: Most or full conceptual match and similarity
- Key criteria: Concept should match, not exact wording.
"""

# Define output schema for the LLM judge
class Grade(BaseModel):
    score: bool = Field(description="Boolean that indicates whether the response is accurate relative to the reference answer")

# Define LLM judge that grades the accuracy of the response relative to reference output
def accuracy(outputs: dict, reference_outputs: dict) -> bool:
  
  response = openai_client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
      { "role": "system", "content": instructions },
      { "role": "user", "content": f"""Ground Truth answer: {reference_outputs["answer"]}; 
      Student's Answer: {outputs["response"]}"""
  }],
    response_format=Grade
  );
  return response.choices[0].message.parsed.score

In [69]:
# After running the evaluation, a link will be provided to view the results in langsmith
experiment_results = client.evaluate(
    target,
    data = "Sample dataset",
    evaluators = [
        accuracy,
        # can add multiple evaluators here
    ],
    experiment_prefix = "first-eval-in-langsmith",
    max_concurrency = 2,
)

View the evaluation results for experiment: 'first-eval-in-langsmith-532bfb7b' at:
https://smith.langchain.com/o/6fe80679-05e1-4b72-908a-9af5dfa86dd5/datasets/83fb0beb-9b1c-435c-ae28-cbb9cec1f593/compare?selectedSessions=2426e413-a5c9-4cc9-ae69-ef69d630e2da




0it [00:00, ?it/s]Error running evaluator <DynamicRunEvaluator accuracy> on run c425ef2d-050b-4a3e-a595-81a00f7baf25: KeyError('answer')
Traceback (most recent call last):
  File "c:\Users\newac\OneDrive\Desktop\Master\.venv\Lib\site-packages\langsmith\evaluation\_runner.py", line 1573, in _run_evaluators
    evaluator_response = evaluator.evaluate_run(
                         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\newac\OneDrive\Desktop\Master\.venv\Lib\site-packages\langsmith\evaluation\evaluator.py", line 331, in evaluate_run
    result = self.func(
             ^^^^^^^^^^
  File "c:\Users\newac\OneDrive\Desktop\Master\.venv\Lib\site-packages\langsmith\run_helpers.py", line 617, in wrapper
    raise e
  File "c:\Users\newac\OneDrive\Desktop\Master\.venv\Lib\site-packages\langsmith\run_helpers.py", line 614, in wrapper
    function_result = run_container["context"].run(func, *args, **kwargs)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users